# MSII Solution Process

## About:
The Design Project Cadet Handout allows for "a spreadsheet, computer program (e.g., MatLab) or hand calculations to support your analysis," provided that your final submission is "formatted in such a way that the instructor can see the formulas you are using and the final answers." Python is a computational tool of a very similar caliber to MatLab (in the hands of an experienced user), and is thus authorized. A brief in-class discussion with Gen. Thompson confirmed this.

I have, to this end, written a fairly comprehensive software library which is capable of computing, among other things, the percent coverage, budget calculations, orbit positions by time, viability assessments, and many other rudimentary computations of a given constellation. The exact code used for the entire project will be included in the submission, as well as a condensed version with the exact formulas we used. 

Understand that the code in this notebook will be extremely abstracted and surface level because it relies on the software library (entirely written by me) for all the computations that would truly be considered Astro-related.



## MS II Checklist:
### Pre-Checklist Constellation Computations:

#### Import all the necessary code from our custom software library:

In [22]:
# handle all the necessary imports from our custom software library
from MSII_Components import *
from MSII_useful_consts import *
from MSII_Constellation import MSII_Constellation
from MSII_Orbit import MSII_Orbit
from MSII_Constraints import *
from Formula_Sheet import *

#### Create a modifyable template of our base constellation (already has good coverage):

In [23]:
'''
NOTE: We are no longer using the MSI constellation template because we have determined that it is bad. We came up with a new one.
for further details, please see MSII_Base_Orbit_Determination.ipynb, where we did all the computations and reasoning.
'''

# def create_MSI_constellation(payload, adcs, structure):
#     msI_a = GEOSYNCHRONOUS_SEMIMAJOR_AXIS # 42241km from MSI
#     max_height = get_payload_max_height(payload, MIN_RESOLUTION) # how high can we get and still hit our
#     # target resolution (formula derivation done and included on paper) in km

#     # from formula sheet: Ra = a(1 + e)
#     # it follows: e = (Ra / a) - 1 = ((h + Rearth) / a) - 1
#     max_eccentricity = ((max_height + R_EARTH) / msI_a) - 1
#     max_eccentricity = abs(max_eccentricity)
#     max_eccentricity = 0.834

#     return { 
#         "payload": payload,
#         "adcs": adcs,
#         "structure": structure,
#         "sats": [ # we update all the fields that will change with new MSII requirements (just e)
#             {
#                 "a": msI_a,
#                 "e": max_eccentricity,
#                 "i": 50.825, # from MSI
#                 "raan": 340.5, # from MSI
#                 "w": 245, # from MSI
#                 "v": 180 # from MSI
#             }, {
#                 "a": msI_a,
#                 "e": max_eccentricity,
#                 "i": 50.825, # from MSI
#                 "raan": 100.5, # from MSI
#                 "w": 245, # from MSI
#                 "v": 154.5099795 # from MSI
#             }, {
#                 "a": msI_a,
#                 "e": max_eccentricity,
#                 "i": 50.825, # from MSI
#                 "raan": 220.5, # from MSI
#                 "w": 245, # from MSI
#                 "v": 205.4900205 # from MSI
#             }
#         ]
#     }

# take a period idea, turn it into a basic constellation
def create_geosync_constellation(idea_payload=PAYLOADS[0], idea_adcs=ADCS[2], idea_struct=STRUCTURES[2]) -> MSII_Constellation:
    idea = ["Geo-Sync Perigee", h_to_sec(24)]

    altitude_offset = get_payload_max_height(idea_payload, MIN_RESOLUTION) - (MIN_DRAGLESS_ALTITUDE + 1) # +1 to prevent rounding errors
    idea_period = idea[1] # store the target period of the idea
    idea_max_h = get_payload_max_height(idea_payload, MIN_RESOLUTION) # how high can we approach from?
    
    idea_a = period_to_axis(idea_period) # calculate a from period (in seconds)
    idea_i = get_avg_latitude(POINTS_OF_INTEREST) # average out the inclinations of each point
    if (idea_i < 0):
        idea_i += 180 # allow for points south of the equator, should not be a problem
    idea_e = perigee_h_to_e(idea_a, idea_max_h - altitude_offset, False) # what is our max eccentricity to get that height?

    idea_w = 90 # we want to set argument of perigee to 90 to simplify calculations
    if (get_avg_latitude(POINTS_OF_INTEREST) < 0):
        idea_w = 270 # allow for points south of the equator, should not be a problem
    
    idea_sat_coes = []
    for sat_num in range(NUM_SATS):
        time_on_target = (idea_period / NUM_SATS) * sat_num # stagger the times on target evenly
        
        sat = {}
        sat["a"] = idea_a # initialize all the variables that stay the same accross sats
        sat["e"] = idea_e
        sat["w"] = idea_w
        sat["i"] = idea_i
        
        # now for the hard bit, we need to change raan and v so they each hit apogee over
        # the target points at staggered times
        avg_long = get_avg_longitude(POINTS_OF_INTEREST)
        # the raan that would put us over the points in BORG at t=0
        avg_raan_t_0 = avg_long - INTIAL_EARTH_POS - idea_w
        # the raan that would put us over the points in BORG at time on target
        avg_raan_t = avg_raan_t_0 + ((360 / DAY_AS_SECONDS) * time_on_target) # assume 24 hour Earth period
        sat["raan"] = avg_raan_t # set that as our raan
        sat["v"] = calc_v_at_t0_to_hit_perigee_at_t(idea_a, idea_e, time_on_target)
        idea_sat_coes.append(sat) # add the satellite coes to our list
    
    assert(len(idea_sat_coes) == NUM_SATS) # make sure we have the right amount of sats
    
    idea_constellation_dict = {
        "structure": idea_struct, # use the best structure by default so it fits the camera for sure
        "adcs": idea_adcs, # best adcs by default
        "payload": idea_payload, # same with the imager
        "sats": idea_sat_coes
    }
    
    return idea_constellation_dict # wrap it all nicely in a constellation dictionary

#### Generate all possible hardware configurations

In [24]:
constellation_options = []
for payload in PAYLOADS: # find every possible payload-adcs-structure combination
    for adcs in ADCS:
        for struct in STRUCTURES:
            # apply our template from before (which includes modified e computations)
            constellation_object = MSII_Constellation(create_geosync_constellation(payload, adcs, struct))
            constellation_options.append(constellation_object)

# output how many we found
print(f"Found {len(constellation_options)} possible constellations.")

viable_constellation_options = []
for constellation in constellation_options:
    # filter out all the ones that obviously fail (i.e. out of budget, crash into earth, fail volume constraints)
    if (constellation.get_concise_summary().passes_inspection):
        viable_constellation_options.append(constellation)
    # print(constellation.assess_orbit_viability().to_string())

# output how many we found
print(f"Found {len(viable_constellation_options)} viable constellation options")

Found 36 possible constellations.
Found 24 viable constellation options


#### Figure out which one has the best coverage
Note that the algorithm for this is complicated and is included in the algorithm explanation portion of the project.

In [25]:
# we are having slight issues with our coverage computation algorithm, probably something to do with the
# fact that the Earth position and the orbits are out of sync 

best_coverage = 0 # assume zero coverage to start, update later
best_constellation = None

for constellation in viable_constellation_options:
    constellation : MSII_Constellation
    coverage_report = constellation.assess_coverage()
    if (not coverage_report.adcs_cam_compatible):
        # our pointing accuracy is so bad we cannot reliably point at anything
        # therefore, do not count this option's coverage in the contest
        continue

    if (coverage_report.percent_coverage > best_coverage):
        best_coverage = coverage_report.percent_coverage
        best_constellation = constellation
    elif (coverage_report.percent_coverage == best_coverage):
        # if they have the same coverage but one is cheaper, choose that one
        if (best_constellation.assess_budget().cost > constellation.assess_budget().cost):
            best_coverage = coverage_report.percent_coverage
            best_constellation = constellation


print(f"Found constellation with {best_coverage}% coverage")

# output the constellation
print(best_constellation.to_string("#1"))
print(best_constellation.get_comprehensive_summary().to_string("#1"))




Found constellation with 0.06425% coverage
Constellation #1:
	Payload:   Astra Hi-Res Imager
	Structure: Option III
	ADCS:      Option II
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 290.375
		w    : 90
		v    : 0.0
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 50.375
		w    : 90
		v    : 146.44029712684926
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 170.375
		w    : 90
		v    : 169.65931160912885

Constellation #1 Comprehensive Summary: 
	Constellation #1 Concise Summary:
		Passes:       True
		In Budget:    True
		Viable Orbit: True
		Volume Fits:  True
	
	Constellation #1 Orbit Viability: VIABLE
		ORBIT 1
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      71124.92146028765 km
			Min Altitude:      600.9999999999991 km
			Is Geosynchronous: True
		
		ORBIT 2
			CA

## Alright! Now that we have our orbit, we can prove it against the checklist.